<a href="https://colab.research.google.com/github/Saheer7/Pyspark/blob/master/4_Spark_Data_Frame_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 48kB/s 
     |████████████████████████████████| 204kB 36.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=2e5584c8ea78d90424b6032f9495c43189110c5a26385e955daeaf3e8ab903f7
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession    #Starting a Spark session

In [4]:
spark = SparkSession.builder.appName("Basics").getOrCreate()